In [1]:
from IPython.core.display import display, HTML
from h5py import File
import pandas as pd
import numpy as np
from pathlib import Path
import os
root_path = Path(r"C:\Users\robin\Documents\Cours\Poly\Recherche\FIBI_Recherche")
print(root_path)
os.chdir(str(root_path))
from FIBI.data.analyse_cpp_results.clustering.extract_infos import parse_key, parse_metadata, DataCategory, corresp_impr_class
import json

C:\Users\robin\AppData\Local\Temp\ipykernel_13568\1378062725.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


C:\Users\robin\Documents\Cours\Poly\Recherche\FIBI_Recherche


## Load data

In [2]:
data = []
with File("data/table1_impr_exp_clust.hdf5", "r") as f:
    for k,v in f["metadata"].items():
        dico_key = parse_key(k)
        v = np.copy(v)
        dico_value = {"key_dataset":k,**parse_metadata(v)}
        data.append(dico_value)
df = pd.DataFrame(data)


In [3]:
display(df.head())

,key_dataset,NUM_POINTS,NUM_DIM,POINTS_DISTR,NUM_CLUST,CLUST_IMPR,IMPR_CLASS,IT_ORDER,SEED_POINTS,SEED_ASSIGN,init_cost,final_cost,num_iter,num_iter_glob,duration
0,"0,20,0,0,0,2,1,1",20,2,0,2,kmeans+,BICL,CURR,0,0,1.0,0.0,0,18556,0.014007
1,"1,20,0,1,0,2,1,1",20,2,0,2,kmeans+,BICL,CURR,1,1,1.0,1.0,1,21213,0.015034
2,"10,20,0,10,0,2,1,1",20,2,0,2,kmeans+,BICL,CURR,10,10,1.0,10.0,10,20804,0.013450
3,"100,20,0,100,0,2,1,1",20,2,0,2,kmeans+,BICL,CURR,100,100,1.0,100.0,100,14462,0.014442
4,"1000,20,0,1000,1,2,1,1",20,2,0,2,kmeans+,FICL,CURR,1000,1000,1.0,1000.0,1000,27042,0.026680


## Verify that we have all of the required results

In [4]:
def check_print(cond:bool):
    if cond:
        print("✅")
    else:
        print("❌")

Number of points:

Expected: [20,30,40,50,60,70,80,90,100,110,120,130,140,150,200,250,300,350,400,450,500,600,700,800,900,1000]

Found:

In [5]:
expected = [20,30,40,50,60,70,80,90,100,110,120,130,140,150,200,250,300,350,400,450,500,600,700,800,900,1000]
num_points = list(df[str(DataCategory.NUM_POINTS)].unique())
num_points.sort()
print(num_points)
check_print(set(num_points) == set(expected))

[20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 200, 250, 300, 350, 400, 450, 500, 600, 700, 800, 900, 1000]
✅


Number of different point and assign seed per number of point x algorithm

Expected:

- for n in [20, 30, ..., 150] : 1000
- for n in [200, 250, ..., 500] : 100
- for n in [500, 600, ..., 1000] : 100

Found:

In [6]:
groups_counts = df.groupby([str(DataCategory.NUM_POINTS),str(DataCategory.IMPR_CLASS),str(DataCategory.NUM_CLUST)]).count().reset_index()
num_points = [20,30,40,50,60,70,80,90,100,110,120,130,140,150,200,250,300,350,400,450,500,600,700,800,900,1000]
expected = {n: 1000 if n<=150 else 100 for n in num_points}
cond = True

for n in expected:
    for impr_class in corresp_impr_class.values():
        df_tmp = groups_counts.query(f"{str(DataCategory.NUM_POINTS)} == {n} and {str(DataCategory.IMPR_CLASS)} == '{impr_class}'")
        if len(df_tmp) == 0:
            print(f"{n} {impr_class}")
            cond = False
            break
        cond = cond and df_tmp.iloc[0][str(DataCategory.SEED_POINTS)] == expected[n]
        cond = cond and df_tmp.iloc[0][str(DataCategory.SEED_ASSIGN)] == expected[n]
check_print(cond)
print(len(df))

✅
178800


## Statistics

### Computed statistics

In [7]:
df_table1_src = df[[
    str(DataCategory.NUM_POINTS),
    str(DataCategory.NUM_CLUST),
    str(DataCategory.IMPR_CLASS),
    str(DataCategory.final_cost),
    str(DataCategory.num_iter),
    str(DataCategory.num_iter_glob),
    str(DataCategory.duration),
    ]]
df_table1 = df_table1_src.copy().groupby([str(DataCategory.NUM_POINTS),str(DataCategory.NUM_CLUST),str(DataCategory.IMPR_CLASS)]).mean().reset_index()
df_table1 = df_table1.query(f"{str(DataCategory.IMPR_CLASS)} != 'BICL'")
df_table1.sort_values(by=[str(DataCategory.NUM_POINTS),str(DataCategory.NUM_CLUST),str(DataCategory.IMPR_CLASS)][::-1], inplace=True)
df_table1.reset_index(drop=True, inplace=True)
df_table1_str = df_table1.to_html(formatters={
    str(DataCategory.NUM_POINTS): '{:d}'.format,
    str(DataCategory.NUM_POINTS): '{:d}'.format,
    str(DataCategory.IMPR_CLASS): '{:s}'.format,
    str(DataCategory.final_cost): '{:.2f}'.format,
    str(DataCategory.num_iter): '{:.0f}'.format,
    str(DataCategory.num_iter_glob): '{:.0f}'.format,
    str(DataCategory.duration): '{:.4e}'.format,
})
display(HTML(df_table1_str))

,NUM_POINTS,NUM_CLUST,IMPR_CLASS,final_cost,num_iter,num_iter_glob,duration
0,20,2,FICL,1499.50,1500,20185,1.6864e-02
1,30,2,FICL,17499.50,17500,31301,2.7054e-02
2,40,2,FICL,33499.50,33500,42472,3.7237e-02
3,50,2,FICL,49499.50,49500,54049,4.7446e-02
4,60,2,FICL,65499.50,65500,64831,5.7700e-02
5,70,2,FICL,81499.50,81500,75906,6.7847e-02
6,80,2,FICL,97499.50,97500,87118,7.8311e-02
7,90,2,FICL,113499.50,113500,98298,8.8475e-02
8,100,2,FICL,129499.50,129500,109145,9.8752e-02
9,110,2,FICL,145499.50,145500,120889,1.0987e-01


## Export results for visualization

In [8]:
computed = df_table1_src.copy()

computed = computed[["NUM_POINTS","NUM_CLUST","IMPR_CLASS","final_cost","num_iter","duration"]]
computed["source"] = "computed"
new_df = computed
display(new_df)

,NUM_POINTS,NUM_CLUST,IMPR_CLASS,final_cost,num_iter,duration,source
0,20,2,BICL,0.0,0,0.014007,computed
1,20,2,BICL,1.0,1,0.015034,computed
2,20,2,BICL,10.0,10,0.013450,computed
3,20,2,BICL,100.0,100,0.014442,computed
4,20,2,FICL,1000.0,1000,0.026680,computed
...,...,...,...,...,...,...,...
178795,80,4,FICL,99995.0,99995,0.034305,computed
178796,80,4,FICL,99996.0,99996,0.031999,computed
178797,80,4,FICL,99997.0,99997,0.029851,computed
178798,80,4,FICL,99998.0,99998,0.029374,computed


In [9]:
def q1(x):
    return x.quantile(0.25)

def median(x):
    return x.quantile(0.50)
def q3(x):
    return x.quantile(0.75)
f = ["mean","std","min",q1,median,q3,"max"]
df_avg = new_df.groupby(["NUM_POINTS","NUM_CLUST","IMPR_CLASS","source"]).agg(f)
df_avg.reset_index(inplace=True)
df_avg.fillna(-1,inplace=True)
Lout = []
for elem in df_avg.to_dict(orient="records"):
    dico = {}
    for k,v in elem.items():
        if k[1] == '':
            dico[k[0]] = v
        else:
            if k[0] not in dico:
                dico[k[0]] = {}
            dico[k[0]][k[1]] = v
    Lout.append(dico)
dico_filtered = {}
for n in new_df["NUM_POINTS"].unique():
    for nClust in new_df["NUM_CLUST"].unique():
        for impr_class in new_df["IMPR_CLASS"].unique():
            tmp_dico = new_df.query(f"NUM_POINTS == {n} and NUM_CLUST == {nClust} and IMPR_CLASS == '{impr_class}' and source == 'computed'").to_dict(orient="records")
            if len(tmp_dico) == 0:
                continue
            dico_filtered[f"{n}_{impr_class}_{nClust}_computed"] = tmp_dico
            
dico_out = {"grouped":Lout,"filtered":dico_filtered}

with open("data/table1_clust_impr_visualize.json","w") as f:
    json.dump(dico_out,f,indent=2)